In [ ]:
import torch

def format_pytorch_version(version):
    return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
pip uninstall networkx

In [ ]:
pip install networkx==2.8.8

In [ ]:
pip install torchmetrics==0.9.3

---

In [ ]:
import time as tm
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch import nn
from torchmetrics import AUROC, F1Score
from torch_geometric.nn import GATConv
from torch_geometric.utils.convert import from_networkx

In [ ]:
df      = pd.read_csv('y_train.csv')
y_train = torch.tensor(df['default']).type(torch.FloatTensor)

In [ ]:
G      = nx.read_gpickle("G_area.gpickle")
rs     = random.sample(list(G.nodes), int(0.5 * len(list(G.nodes))))
G.remove_edges_from(list(G.edges(rs)))
data   = from_networkx(G, group_node_attrs = all)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class GAT_GRU(torch.nn.Module):

    def __init__(self, in_channels:int, out_channels:int, heads:int, concat:bool = False):
        super(GAT_GRU, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self._create_parameters_and_layers()

    def _create_update_gate_parameters_and_layers(self):
        self.conv_z = GATConv(in_channels = self.in_channels, out_channels = self.out_channels, heads = self.heads,
                              concat = self.concat)
        self.linear_z = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_reset_gate_parameters_and_layers(self):
        self.conv_r = GATConv(in_channels = self.in_channels, out_channels = self.out_channels, heads = self.heads,
                              concat = self.concat)
        self.linear_r = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_candidate_state_parameters_and_layers(self):
        self.conv_h = GATConv(in_channels = self.in_channels, out_channels = self.out_channels, heads = self.heads,
                              concat = self.concat)
        self.linear_h = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, H):
        Z = torch.cat([self.conv_z(X, edge_index), H], axis = 1)
        Z = self.linear_z(Z)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, H):
        R = torch.cat([self.conv_r(X, edge_index), H], axis = 1)
        R = self.linear_r(R)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, H, R):
        H_tilde = torch.cat([self.conv_h(X, edge_index), H * R], axis = 1)
        H_tilde = self.linear_h(H_tilde)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H

    def forward(self, X:torch.FloatTensor, edge_index:torch.LongTensor, H:torch.FloatTensor = None) -> torch.FloatTensor:
        H = self._set_hidden_state(X, H)
        Z = self._calculate_update_gate(X, edge_index, H)
        R = self._calculate_reset_gate(X, edge_index, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, H, R)
        H = self._calculate_hidden_state(Z, H, H_tilde)
        return H

In [ ]:
class GAT_GRU_NEW(torch.nn.Module):

    def __init__(self, in_channels:int, out_channels:int, heads:int, periods:int, concat:bool = False):
        super(GAT_GRU_NEW, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.periods = periods
        self.concat = concat
        self._setup_layers()

    def _setup_layers(self):
        self._base_gat_gru = GAT_GRU(in_channels = self.in_channels, out_channels = self.out_channels,
                                     heads = self.heads, concat = self.concat)

    def forward(self, X1:torch.FloatTensor, edge_index1:torch.LongTensor,
                      X2:torch.FloatTensor, edge_index2:torch.LongTensor,
                      X3:torch.FloatTensor, edge_index3:torch.LongTensor,
                      X4:torch.FloatTensor, edge_index4:torch.LongTensor,
                      X5:torch.FloatTensor, edge_index5:torch.LongTensor,
                      X6:torch.FloatTensor, edge_index6:torch.LongTensor,
                      H:torch.FloatTensor = None) -> torch.FloatTensor:

        for period in range(self.periods):
            if period == 0:
                X = X1
                edge_index = edge_index1
                H = self._base_gat_gru(X, edge_index, H)
            elif period == 1:
                X = X2
                edge_index = edge_index2
                H = self._base_gat_gru(X, edge_index, H)
            elif period == 2:
                X = X3
                edge_index = edge_index3
                H = self._base_gat_gru(X, edge_index, H)
            elif period == 3:
                X = X4
                edge_index = edge_index4
                H = self._base_gat_gru(X, edge_index, H)
            elif period == 4:
                X = X5
                edge_index = edge_index5
                H = self._base_gat_gru(X, edge_index, H)
            elif period == 5:
                X = X6
                edge_index = edge_index6
                H = self._base_gat_gru(X, edge_index, H)
        return H

In [ ]:
class RecurrentGAT(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(RecurrentGAT, self).__init__()
        self.recurrent = GAT_GRU_NEW(node_features, --, --, periods)
        self.linear_1 = torch.nn.Linear(--, --)
        self.dropout = torch.nn.Dropout(p = --)
        self.linear_2 = torch.nn.Linear(--, --)

    def forward(self, x1, edge_index1,
                      x2, edge_index2,
                      x3, edge_index3,
                      x4, edge_index4,
                      x5, edge_index5,
                      x6, edge_index6):
        h = self.recurrent(x1, edge_index1,
                           x2, edge_index2,
                           x3, edge_index3,
                           x4, edge_index4,
                           x5, edge_index5,
                           x6, edge_index6)
        h = self.linear_1(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.linear_2(h)
        return torch.sigmoid(h)

model = RecurrentGAT(node_features = 16, periods = 6).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
st = tm.time()

model.train()
for epoch in tqdm(range(--)):
    for w in range(13):
        for s in range(6):
            exec("x" + str(s + 1) + " = " + "data" + str(w + s + 1) + ".x.to(device)")
            exec("edge_index" + str(s + 1) + " = " + "data" + str(w + s + 1) + ".edge_index.to(device)")
        y_targ = y_train.to(device)
        y_pred = model(x1, edge_index1,
                       x2, edge_index2,
                       x3, edge_index3,
                       x4, edge_index4,
                       x5, edge_index5,
                       x6, edge_index6)
        cost = criterion(y_pred, y_targ.reshape(-1, 1))
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
    if epoch % 100 == 0:
        print('cost = ', cost.item())
        print('accuracy = ', (torch.round(y_pred) == y_targ.reshape(-1, 1)).sum().item() / len(y_targ))

et = tm.time()
elapsed_time = et - st
print('\ntraining time:', elapsed_time, 'seconds')

In [ ]:
df_test = pd.read_csv('y_test.csv')
y_test  = torch.tensor(df_test['default']).type(torch.FloatTensor)

In [ ]:
G_test      = nx.read_gpickle("G_area_test.gpickle")
rs          = random.sample(list(G_test.nodes), int(0.5 * len(list(G_test.nodes))))
G_test.remove_edges_from(list(G_test.edges(rs)))
data_test   = from_networkx(G_test, group_node_attrs = all)

In [ ]:
for s in range(6):
    exec("x" + str(s + 1) + "_test = " + "data" + str(s + 19) + "_test.x.to(device)")
    exec("edge_index" + str(s + 1) + "_test = " + "data" + str(s + 19) + "_test.edge_index.to(device)")

y_targ_test = y_test

In [ ]:
model.eval()
auroc = AUROC(pos_label = 1)
f1score = F1Score(num_classes = 2, multiclass = True)
with torch.no_grad():
    preds = model(x1_test, edge_index1_test,
                  x2_test, edge_index2_test,
                  x3_test, edge_index3_test,
                  x4_test, edge_index4_test,
                  x5_test, edge_index5_test,
                  x6_test, edge_index6_test).cpu()
    target = y_targ_test.reshape(-1, 1).type(torch.int64)
    auroc_test = auroc(preds, target).item()
    f1score_test = f1score(preds, target).item()
    error_test = criterion(preds, y_targ_test.reshape(-1, 1)).item()

In [ ]:
print('AUC = ', auroc_test)
print('F1-score = ', f1score_test)
print('Error = ', error_test)